In [101]:
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
import pandas as pd
import numpy as np

In [102]:
# # Prepare the data data
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target

# # Fit the classifier with max_depth=3
# clf = DecisionTreeClassifier(max_depth=3, random_state=1234)
# model = clf.fit(X, y)

In [103]:
# text_representation = tree.export_text(clf, feature_names=iris.feature_names)
# print(text_representation)

In [104]:
import aux.acessos as ac

In [105]:
query = """

with grupo_mcc as (
select 
	ds_nome_estabelecimento,  
	ds_mcc,
    ds_grupo_mcc, 
	registros 
from (
	select
		ds_nome_estabelecimento, 
		ds_mcc,
        ds_grupo_mcc, 
		registros, 
		rank() over (partition by ds_nome_estabelecimento order by registros desc) as rank
	from
	(
		select
			ds_nome_estabelecimento, 
			ds_mcc,
            ds_grupo_mcc, 
			count(*) as registros
		from backoffice_curated_zone.transaction
		group by 1,2,3
	)) where rank = 1
)
------FIM WITH --------
select  
g.ds_mcc,
g.ds_grupo_mcc,
t.ds_classificacao_nivel_2,
t.ds_classificacao_nivel_3,
t.ds_classificacao_nivel_4
	from platform_curated_zone.estabelecimentos_tratados_classificados t
left join grupo_mcc g 
	on (replace(replace(upper(t.ds_nome_estabelecimento), '*', ''),' ', '')  = replace(replace(upper(g.ds_nome_estabelecimento), '*', ''),' ', ''))
where registros >= 5

"""

In [106]:
df = ac.df_athena('flavia-costa', query)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [6]:
len(df)

150896

In [8]:
df_assoc = df[['ds_mcc','ds_classificacao_nivel_2']]

In [107]:
df_dm = pd.get_dummies(df, columns = df.columns)

In [9]:
df_dm.head()

,ds_mcc_ACADEMIAS / CLUBES,ds_mcc_ACESSORIOS FEMININOS,ds_mcc_AGENCIAS DE VIAGEM / OPERADORAS DE TURISMO (GDS,ds_mcc_AGÊNCIAS DE CONSULTA DE CRÉDITO AO CONSUMIDOR,ds_mcc_ALUGUEL DE EQUIPAMENTO E MOBÍLIA DE ESCRITÓRIOS,"ds_mcc_APOSTAS, CASAS LOTERICAS, BINGOS",ds_mcc_ARMARINHOS E TECIDO,ds_mcc_ASSINATURA CONTINUA,ds_mcc_ASSOCIAÇÕES CÍVICAS E SOCIAIS,ds_mcc_ATACADISTA,...,ds_classificacao_nivel_4_SAPATOS,ds_classificacao_nivel_4_SAUDE PET,ds_classificacao_nivel_4_SEGURO,ds_classificacao_nivel_4_SHOWS E EVENTOS,ds_classificacao_nivel_4_SUPERMERCADOS,ds_classificacao_nivel_4_TAXIS,ds_classificacao_nivel_4_TV A CABO,ds_classificacao_nivel_4_UNIVERSIDADE,ds_classificacao_nivel_4_VAREJISTAS,ds_classificacao_nivel_4_VIDEO
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
import aux.aed as aed

In [11]:
drop_col = []

aed.find_correl(df, drop_col, 0.80)

ds_mcc_Caixa Eletrônico ---> ds_grupo_mcc_SAQUES -> correlacao de 0.8070688216216907
__________________________________________
ds_mcc_Clubes / Atletismo - Sócios ---> ds_classificacao_nivel_3_ACADEMIAS -> correlacao de 0.874619595584125
__________________________________________
ds_mcc_Clubes / Atletismo - Sócios ---> ds_classificacao_nivel_4_ACADEMIAS -> correlacao de 0.874619595584125
__________________________________________
ds_mcc_Cosméticos ---> ds_grupo_mcc_CUIDADOS PESSOAIS -> correlacao de 0.8791459888537467
__________________________________________
ds_mcc_Drogarias e Farmácias ---> ds_grupo_mcc_SAUDE -> correlacao de 0.9792073497049928
__________________________________________
ds_mcc_Drogarias e Farmácias ---> ds_classificacao_nivel_2_SAUDE -> correlacao de 0.8939564118999234
__________________________________________
ds_mcc_Drogarias e Farmácias ---> ds_classificacao_nivel_3_FARMACIAS E DROGARIAS -> correlacao de 0.920107879183772
_________________________________________

In [12]:
# X = np.array(df_dm[df_dm.drop('ds_classificacao_nivel_4_ELETRONICOS', axis = 1).columns]).reshape(1, -1)

In [13]:
# y = np.array(df_dm['ds_classificacao_nivel_4_ELETRONICOS']).reshape(1, -1)

In [14]:
# clf = DecisionTreeClassifier(max_depth=5, random_state=1234)
# model = clf.fit(X, y)

In [15]:
# text_representation = tree.export_text(clf, decimals=0)
# print(text_representation)

In [11]:
transactions = []
for i in range(0, df_assoc.shape[0]):
    transactions.append([str(df_assoc.values[i, j]) for j in range(0, 2)])

print(transactions[0])

['LANCHONETES', 'SERVICOS DE ALIMENTACAO']


In [14]:
# !pip install apyori

In [15]:
from apyori import apriori

In [38]:
rules = apriori(transactions, min_support = 0.01, min_confidence = 0.10, min_lift = 1, min_length = 2)

In [39]:
results = list(rules)

In [40]:
results = pd.DataFrame(results)
results.head(30)

,items,support,ordered_statistics
0,(Fax / Chamadas Locais e Longa Distância),0.406512,"[((), (Fax / Chamadas Locais e Longa Distância..."
1,(TELECOM),0.424551,"[((), (TELECOM), 0.4245506839147492, 1.0)]"
2,"(DELIVERY, Lanchonetes)",0.010829,"[((DELIVERY), (Lanchonetes), 0.431248350488255..."
3,"(SAUDE, Drogarias e Farmácias)",0.043361,"[((Drogarias e Farmácias), (SAUDE), 0.96333922..."
4,"(E-COMMERCE, Vendas de Roupas para Família)",0.027907,"[((E-COMMERCE), (Vendas de Roupas para Família..."
5,"(EDUCACAO, Software)",0.015898,"[((EDUCACAO), (Software), 0.8126693766937668, ..."
6,"(Fax / Chamadas Locais e Longa Distância, TELE...",0.403894,"[((), (Fax / Chamadas Locais e Longa Distância..."
7,"(Lojas de Departamento, LOJAS DE DEPARTAMENTO)",0.020948,"[((LOJAS DE DEPARTAMENTO), (Lojas de Departame..."
8,"(LOJAS DE DEPARTAMENTO, Vendas de Roupas para ...",0.013142,"[((LOJAS DE DEPARTAMENTO), (Vendas de Roupas p..."
9,"(SERVICOS DE ALIMENTACAO, Lanchonetes)",0.026840,"[((Lanchonetes), (SERVICOS DE ALIMENTACAO), 0...."


In [41]:
def count_words_cluster(df, termo, coluna_cluster):
    contagemClusters = pd.DataFrame(columns=['index', 'count', 'cluster'])
    clusters = df[coluna_cluster].unique()
    for i in clusters:
        filtro_cluster = df[df[coluna_cluster] == i]
        newdf = filtro_cluster[termo].str.split(expand=True).stack().value_counts().reset_index(name="count")
        newdf['cluster'] = i
        contagemClusters = contagemClusters.append(newdf)
    return contagemClusters

In [52]:
import re
import string 
import unidecode
from numpy.core.defchararray import find
from numpy.core.defchararray import replace

In [53]:
def text_preproc(x):
    if x is not None:
        x = x.upper()
        x = unidecode.unidecode(x)
        x = x.replace(","," ")
        x = re.sub(r'\s', ' ', x)
        x = re.sub(r'\r', ' ', x)
        x = re.sub(r'\t', ' ', x)
        x = re.sub(r"'", ' ', x)
        x = re.sub(r'/[^a-zA-Z0-9 ]/g', ' ', x)
        x = re.sub(r'/[^\w\s]/gi', '', x)
        x = x.replace('     ',' ')
        x = x.replace('   ',' ')
        x = x.replace('  ',' ')
        x = x.strip()

        return x
    
    else:
        return None

In [54]:
df['ds_mcc'] = df['ds_mcc'].apply(text_preproc)

In [98]:
df_cont = count_words_cluster(df, 'ds_mcc', 'ds_classificacao_nivel_2' )
df_cont = df_cont.query('count > 50').sort_values(['cluster', 'count'], ascending=[True, False]).groupby('index').agg({'cluster':['first'],'count':['first'] })
df_cont = df_cont.reset_index()
df_cont = df_cont.sort_values(('count', 'first'), ascending = False)

retirar = ["da", "de", "o", "a", "outros", "em", "do", "na", "nas", "/", "e", "E", "DE", "DA", "NAO", "PARA"]
df_cont_final = df_cont[~df_cont['index'].isin(retirar)]
df_cont_final.head(20)


<ipython-input-41-02694267d02d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contagemClusters = contagemClusters.append(newdf)
<ipython-input-41-02694267d02d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contagemClusters = contagemClusters.append(newdf)
<ipython-input-41-02694267d02d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contagemClusters = contagemClusters.append(newdf)
<ipython-input-41-02694267d02d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contagemClusters = contagemClusters.append(newdf)
<ipython-input-41-02694267d02d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

,index,cluster,count
,,first,first
141,ROUPAS,E-COMMERCE,4495
167,VENDAS,E-COMMERCE,4221
79,FAMILIA,E-COMMERCE,4211
147,SOFTWARE,EDUCACAO,2399
98,LANCHONETES,DELIVERY,1744
20,AUDIOVISUAL,ENTRETENIMENTO,1327
113,MIDIA,ENTRETENIMENTO,1327
158,TELECOMUNICACOES,TELECOM,1316
123,PAGAMENTOS,FINANCAS,1216


In [99]:
len(df_cont_final)

165

In [90]:
df_cont.sort_values(('count', 'first'), ascending = False).head(20)

,cluster,count
,first,first
index,,
FAMILIA,E-COMMERCE,4211
SOFTWARE,EDUCACAO,2399
LANCHONETES,DELIVERY,1744
AUDIOVISUAL,ENTRETENIMENTO,1327
MIDIA,ENTRETENIMENTO,1327
DEPARTAMENTO,E-COMMERCE,931
SAPATOS,E-COMMERCE,701
RESTAURANTES,DELIVERY,665


In [82]:
df_cont.tail()

,cluster,count
,first,first
index,,
VIDRO,FINANCAS,1
VOCACIONAL,EDUCACAO,129
WEBSERVICE,TELECOM,1
ZOOLOGICO,ENTRETENIMENTO,31
[EXCLUDING,FINANCAS,10


In [92]:
# df_cont["count"] = df_cont.groupby("index")["count"].cumcount()
# df_cont.loc[df_cont["count"]<3].drop(['count'], axis=1)



In [93]:
df_cont_final.head(20)

,index,cluster,count
,,first,first
0,(AGUA,E-COMMERCE,7
1,(CATALOGO,E-COMMERCE,9
2,(CLINICAS,SAUDE,4
3,(EQUESTRE,LOJAS DE DEPARTAMENTO,4
4,(GDS,FINANCAS,4
5,(MANIPULACAO),CUIDADOS PESSOAIS,51
6,(NAO,ENTRETENIMENTO,5
7,(NOVOS,VIAGEM,7
8,(OS),LOJAS DE DEPARTAMENTO,4


In [48]:
df_cont[~df_cont['index'].isin(retirar)].to_csv('termos_mcc.csv')

In [16]:
import aux.perfil as pf

In [20]:
colunas_perfil = ['ds_mcc']

df['cont'] = 1

pf.report_perfil_grupos(df, 'ds_classificacao_nivel_4', 'cont', 0.8, 3, colunas_perfil)

report dos perfis finalizado!


/home/flavia.costa/work/git/codigos/aux/perfil.py:280: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna_contagem] = df[coluna_contagem].astype('int')
/home/flavia.costa/work/git/codigos/aux/perfil.py:388: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  casos_signif_final = casos_signif_final.append(casos_signif).sort_values('diferenca_abs', ascending = False)


In [18]:
# Fit the classifier with max_depth=3

for i in df_dm.columns:   
    print(i)
    X = np.array(df_dm[df_dm.drop(i, axis = 1).columns]).reshape(1, -1)
    y = np.array(df_dm[i]).reshape(1, -1)
    X.shape
    
    clf = DecisionTreeClassifier(max_depth=2, random_state=1234)
    model = clf.fit(X, y)

    text_representation = tree.export_text(clf)

    print(text_representation)
    print("_________________")

ds_mcc_ACADEMIAS / CLUBES
|--- class: 0

_________________
ds_mcc_ACESSORIOS FEMININOS
|--- class: 0

_________________
ds_mcc_AGENCIAS DE VIAGEM / OPERADORAS DE TURISMO (GDS
|--- class: 0

_________________
ds_mcc_AGÊNCIAS DE CONSULTA DE CRÉDITO AO CONSUMIDOR
|--- class: 0

_________________
ds_mcc_ALUGUEL DE EQUIPAMENTO E MOBÍLIA DE ESCRITÓRIOS
|--- class: 0

_________________
ds_mcc_APOSTAS, CASAS LOTERICAS, BINGOS
|--- class: 0

_________________
ds_mcc_ARMARINHOS E TECIDO
|--- class: 0

_________________
ds_mcc_ASSINATURA CONTINUA
|--- class: 0

_________________
ds_mcc_ASSOCIAÇÕES CÍVICAS E SOCIAIS
|--- class: 0

_________________
ds_mcc_ATACADISTA
|--- class: 0

_________________
ds_mcc_AZUL AIR
|--- class: 0

_________________
ds_mcc_Acessórios Femininos
|--- class: 0

_________________
ds_mcc_Acompanhante
|--- class: 0

_________________
ds_mcc_Aerolineas Argentinas
|--- class: 0

_________________
ds_mcc_Aeroportos
|--- class: 0

_________________
ds_mcc_Agencias De Consulta 

In [ ]:
for i in df_dm.columns:   
    print(i)
    X = np.array(df_dm[df_dm.drop(i, axis = 1).columns]).reshape(1, -1)
    y = np.array(df_dm[i]).reshape(1, -1)
    X.shape
    
    clf = DecisionTreeClassifier(max_depth=2, random_state=1234)
    model = clf.fit(X, y)

    text_representation = tree.export_text(clf)

    print(text_representation)
    print("_________________")

In [108]:
df_dm.head()

,ds_mcc_ACADEMIAS / CLUBES,ds_mcc_ACESSORIOS FEMININOS,ds_mcc_AGENCIAS DE VIAGEM / OPERADORAS DE TURISMO (GDS,ds_mcc_AGÊNCIAS DE CONSULTA DE CRÉDITO AO CONSUMIDOR,ds_mcc_ALUGUEL DE EQUIPAMENTO E MOBÍLIA DE ESCRITÓRIOS,"ds_mcc_APOSTAS, CASAS LOTERICAS, BINGOS",ds_mcc_ARMARINHOS E TECIDO,ds_mcc_ASSINATURA CONTINUA,ds_mcc_ASSOCIAÇÕES CÍVICAS E SOCIAIS,ds_mcc_ATACADISTA,...,ds_classificacao_nivel_4_SAPATOS,ds_classificacao_nivel_4_SAUDE PET,ds_classificacao_nivel_4_SEGURO,ds_classificacao_nivel_4_SHOWS E EVENTOS,ds_classificacao_nivel_4_SUPERMERCADOS,ds_classificacao_nivel_4_TAXIS,ds_classificacao_nivel_4_TV A CABO,ds_classificacao_nivel_4_UNIVERSIDADE,ds_classificacao_nivel_4_VAREJISTAS,ds_classificacao_nivel_4_VIDEO
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

i = 'ds_mcc_ACADEMIAS / CLUBES'

X = np.array(df_dm[df_dm.drop(i, axis = 1).columns]).reshape(1, -1)
y = np.array(df_dm[i]).reshape(1, -1)
X.shape
    
clf = DecisionTreeClassifier(max_depth=2, random_state=1234)
model = clf.fit(X, y)

text_representation = tree.export_text(clf)

print(text_representation)
print("_________________")

|--- class: 0

_________________
